In [1]:
import cv2
import numpy as np
import pandas as pd
import sklearn.cluster
from PIL import Image
from scipy.stats import mode

In [1]:
!pip install opencv-contrib-python

In [2]:
video_list = ['видео по 1 шаблону пульс 90-80.MOV', 'пульс 154 шаблон 4.MOV', 'пульс 90-80 видео по 2 шаблону.MOV',
              'пульс 116-90.MOV', 'пульс 142-108 3-4 шаблон.MOV', 'пульс 160 шаблон 4.MOV', 
              'спокойное дыхание пульс 90.MOV']

pulse_list = [[80, 90], [150, 156], [80, 90], [90, 116], [120, 142], [158, 162], [88, 92]]

#video_list = ['видео по 1 шаблону пульс 90-80.MOV', 'пульс 154 шаблон 4.MOV']
#pulse_list = [[80, 90], [150, 156]]

X_train = pd.DataFrame()

In [3]:
def dominant_colors(image): # PIL image input

    num_clusters = 10

    ar = np.asarray(image)
    shape = ar.shape
    ar = ar.reshape(np.product(shape[:2]), shape[2]).astype(float)

    kmeans = sklearn.cluster.KMeans(
                                     n_clusters=num_clusters,
                                     init="k-means++",
                                     max_iter=20,
                                     random_state=1000
                                    ).fit(ar)
    codes = kmeans.cluster_centers_

    vecs, dist = scipy.cluster.vq.vq(ar, codes) # assign codes
    counts, bins = np.histogram(vecs, len(codes)) # count occurrences

    colors = []
    for index in np.argsort(counts)[::-1]:
        colors.append(tuple([int(code) for code in codes[index]]))
    return colors # returns colors in order of dominance

In [3]:
for video, pulse in zip(video_list, pulse_list):
    
    """t_list 	S_list 	s_w"""
    cap = cv2.VideoCapture(video)
    frame_count = 0
    S_list = []
    s_pr = []
    t_list = []
    s_w = []

    while cap.isOpened():
        ret, frame = cap.read()
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        if frame is not None:

            image = cv2.GaussianBlur(frame, (5, 5), cv2.BORDER_DEFAULT)
            original = image.copy()
            hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

            # cv2.imwrite('Снимок.png', hsv)

            hsv_lower = np.array([105, 68, 37])
            hsv_upper = np.array([113, 138, 124])
    
            mask = cv2.inRange(hsv, hsv_lower, hsv_upper)
            result = cv2.bitwise_and(original, original, mask=mask)

            cv2.imshow('mask', mask)

            # apply morphology close to fill interior regions in mask
            kernel = np.ones((7, 7), np.uint8)
            morph = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
            kernel = np.ones((10, 10), np.uint8)
            morph = cv2.morphologyEx(morph, cv2.MORPH_CLOSE, kernel)

            cv2.imshow('morph', morph)

            # counting the number of pixels
            number_of_white_pix = np.sum(morph == 255)
            number_of_black_pix = np.sum(morph == 0)

            #print('Number of white pixels:', number_of_white_pix)
            #print('Number of black pixels:', number_of_black_pix)
            #print(frame_count)
        
            frame_count+=1
            if not(frame_count % 15):
                if frame_count == 15:
                    s_pr.append(number_of_white_pix / (number_of_white_pix + number_of_black_pix))
                    t_list.append(frame_count)
                    s_w.append(number_of_white_pix)

                else:
                    s_pr.append(number_of_white_pix / (number_of_white_pix + number_of_black_pix))
                    delta_s = (s_pr[frame_count % 15] - s_pr[frame_count % 15 - 1]) ** 2
                    S_list.append(delta_s)
                    t_list.append(frame_count)
                    s_w.append(number_of_white_pix)

        else:
            break

    cv2.destroyAllWindows()

    
    DF = pd.DataFrame({'t_list':t_list[1:len(t_list)], 'S_list':S_list, 's_w':s_w[1:len(s_w)]})
    
    """timi_t"""
    tt = 0
    timi_t = np.zeros(len(s_w))
    s_ind_list = []

    for s_ind in range(len(s_w[1:len(s_w) - 2])):
        if s_w[s_ind] < s_w[s_ind + 1]:
            tt += 15
            s_ind_list.append(s_ind)
        else:
            timi_t[s_ind_list] = tt * 1.5 / 30
            tt = 0
            s_ind_list = []

    timi_t[timi_t == 0] = mode(timi_t[timi_t != 0], axis = 0)[0]      
    timi_t[timi_t == max(timi_t)] = np.mean(timi_t)
    
    DF['timi_t'] = timi_t[1:len(timi_t)]
    
    
    #color_feature
    
    """color_feature = list()
    face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')

    Frame = cv2.VideoCapture(video)
    frame_count = 0

    while True:
        
        
        ret, image = Frame.read()
        if image is not None:
            
            grey = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            rects = face_classifier.detectMultiScale(grey, scaleFactor=1.2, minSize=(15, 15))
             # Draw a rectangle around the faces
            for (x, y, w, h) in rects:
                cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cropped = image[y:y+h, x:x+w]
            frame_count += 1
            if not(frame_count % 15):
                color_feature.append((dominant_colors(cropped)[0][0]/sum(dominant_colors(cropped)[0]) +
                             dominant_colors(cropped)[0][1]/sum(dominant_colors(cropped)[0]) +
                             dominant_colors(cropped)[0][2]/sum(dominant_colors(cropped)[0])) /
                             (np.sqrt(dominant_colors(cropped)[0][0]**2 + dominant_colors(cropped)[0][1]**2 +
                             dominant_colors(cropped)[0][2]**2)) * 100
                            )

        # Display the resulting frame
            cv2.imshow('Face Detection on Video', cropped)
        # wait for 'c' to close the application
            if cv2.waitKey(1) & 0xFF == ord('c'):
                break
            Frame.release()
        else:
            break
        
    cv2.destroyAllWindows()
    
    
    DF['face_color_feature'] = color_feature[1:len(color_feature)]"""
    
    """Targets"""
    targets = np.random.randint(pulse[0], high = pulse[1], size = len(s_w))
    DF['target'] = targets[1:len(targets)]

    """Добавление содержимого в X_train"""
    X_train = X_train.append(DF, ignore_index=True)

Number of white pixels: 335624
Number of white pixels: 331189
Number of white pixels: 323818
Number of white pixels: 316385
Number of white pixels: 311999
Number of white pixels: 316349
Number of white pixels: 321533
Number of white pixels: 325567
Number of white pixels: 325656
Number of white pixels: 326722
Number of white pixels: 324592
Number of white pixels: 326010
Number of white pixels: 328868
Number of white pixels: 330964
Number of white pixels: 332318
Number of white pixels: 333813
Number of white pixels: 336852
Number of white pixels: 335618
Number of white pixels: 336887
Number of white pixels: 339400
Number of white pixels: 341050
Number of white pixels: 336590
Number of white pixels: 327048
Number of white pixels: 319591
Number of white pixels: 318238
Number of white pixels: 321884
Number of white pixels: 314321
Number of white pixels: 292159
Number of white pixels: 287971
Number of white pixels: 294399
Number of white pixels: 297831
Number of white pixels: 300187
Number o

Number of white pixels: 195379
Number of white pixels: 195060
Number of white pixels: 196628
Number of white pixels: 190980
Number of white pixels: 202521
Number of white pixels: 207145
Number of white pixels: 201915
Number of white pixels: 198356
Number of white pixels: 183924
Number of white pixels: 178249
Number of white pixels: 191935
Number of white pixels: 196356
Number of white pixels: 195497
Number of white pixels: 186302
Number of white pixels: 185561
Number of white pixels: 195978
Number of white pixels: 198073
Number of white pixels: 202441
Number of white pixels: 198674
Number of white pixels: 205211
Number of white pixels: 205603
Number of white pixels: 207951
Number of white pixels: 201938
Number of white pixels: 200708
Number of white pixels: 201611
Number of white pixels: 201557
Number of white pixels: 200425
Number of white pixels: 203201
Number of white pixels: 200080
Number of white pixels: 204130
Number of white pixels: 201854
Number of white pixels: 204030
Number o

Number of white pixels: 367059
Number of white pixels: 356342
Number of white pixels: 342185
Number of white pixels: 324199
Number of white pixels: 303073
Number of white pixels: 293218
Number of white pixels: 291151
Number of white pixels: 303563
Number of white pixels: 304658
Number of white pixels: 331972
Number of white pixels: 313234
Number of white pixels: 315749
Number of white pixels: 311723
Number of white pixels: 313738
Number of white pixels: 314694
Number of white pixels: 310237
Number of white pixels: 311896
Number of white pixels: 305166
Number of white pixels: 305483
Number of white pixels: 311268
Number of white pixels: 310891
Number of white pixels: 308593
Number of white pixels: 309311
Number of white pixels: 315873
Number of white pixels: 318705
Number of white pixels: 307787
Number of white pixels: 310020
Number of white pixels: 311916
Number of white pixels: 311140
Number of white pixels: 312310
Number of white pixels: 303413
Number of white pixels: 311580
Number o

Number of white pixels: 206045
Number of white pixels: 197355
Number of white pixels: 201732
Number of white pixels: 196212
Number of white pixels: 201075
Number of white pixels: 196334
Number of white pixels: 197629
Number of white pixels: 196718
Number of white pixels: 195026
Number of white pixels: 194893
Number of white pixels: 190115
Number of white pixels: 196287
Number of white pixels: 194990
Number of white pixels: 196490
Number of white pixels: 194430
Number of white pixels: 194903
Number of white pixels: 189050
Number of white pixels: 191729
Number of white pixels: 191732
Number of white pixels: 199660
Number of white pixels: 198626
Number of white pixels: 195713
Number of white pixels: 191125
Number of white pixels: 181500
Number of white pixels: 190346
Number of white pixels: 190893
Number of white pixels: 187289
Number of white pixels: 188583
Number of white pixels: 188592
Number of white pixels: 186212
Number of white pixels: 187620
Number of white pixels: 191265
Number o

Number of white pixels: 238394
Number of white pixels: 240875
Number of white pixels: 241603
Number of white pixels: 256499
Number of white pixels: 255328
Number of white pixels: 252738
Number of white pixels: 253663
Number of white pixels: 249940
Number of white pixels: 243054
Number of white pixels: 258771
Number of white pixels: 253308
Number of white pixels: 248843
Number of white pixels: 246687
Number of white pixels: 253064
Number of white pixels: 258157
Number of white pixels: 253980
Number of white pixels: 250250
Number of white pixels: 255093
Number of white pixels: 256468
Number of white pixels: 259752
Number of white pixels: 264943
Number of white pixels: 274940
Number of white pixels: 280346
Number of white pixels: 307992
Number of white pixels: 299512
Number of white pixels: 308989
Number of white pixels: 307156
Number of white pixels: 313274
Number of white pixels: 312324
Number of white pixels: 315845
Number of white pixels: 295205
Number of white pixels: 289696
Number o

In [5]:
X_train.to_csv('X_train.csv', index=False)

In [6]:
X_train

,t_list,S_list,s_w,timi_t,target
0,30,0.000006,335624,0.75,87
1,45,0.000022,331189,0.75,81
2,60,0.000068,323818,0.75,85
3,75,0.000140,316385,0.75,88
4,90,0.000194,311999,3.75,82
...,...,...,...,...,...
761,1995,0.000283,279141,0.75,90
762,2010,0.000008,238568,0.75,90
763,2025,0.000496,290418,0.75,90
764,2040,0.000070,261656,0.75,91


# Тестовая выборка

## Считывание данных

In [14]:
video_list = ['Видео для теста.MOV']
X_test = pd.DataFrame()

In [15]:
for video in video_list:
    
    """t_list 	S_list 	s_w"""
    cap = cv2.VideoCapture(video)
    frame_count = 0
    S_list = []
    s_pr = []
    t_list = []
    s_w = []

    while cap.isOpened():
        ret, frame = cap.read()
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        if frame is not None:

            image = cv2.GaussianBlur(frame, (5, 5), cv2.BORDER_DEFAULT)
            original = image.copy()
            hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

            # cv2.imwrite('Снимок.png', hsv)

            hsv_lower = np.array([105, 68, 37])
            hsv_upper = np.array([113, 138, 124])
    
            mask = cv2.inRange(hsv, hsv_lower, hsv_upper)
            result = cv2.bitwise_and(original, original, mask=mask)

            cv2.imshow('mask', mask)

            # apply morphology close to fill interior regions in mask
            kernel = np.ones((7, 7), np.uint8)
            morph = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
            kernel = np.ones((10, 10), np.uint8)
            morph = cv2.morphologyEx(morph, cv2.MORPH_CLOSE, kernel)

            cv2.imshow('morph', morph)

            # counting the number of pixels
            number_of_white_pix = np.sum(morph == 255)
            number_of_black_pix = np.sum(morph == 0)

            #print('Number of white pixels:', number_of_white_pix)
            #print('Number of black pixels:', number_of_black_pix)
            #print(frame_count)
        
            frame_count+=1
            if not(frame_count % 15):
                if frame_count == 15:
                    s_pr.append(number_of_white_pix / (number_of_white_pix + number_of_black_pix))
                    t_list.append(frame_count)
                    s_w.append(number_of_white_pix)

                else:
                    s_pr.append(number_of_white_pix / (number_of_white_pix + number_of_black_pix))
                    delta_s = (s_pr[frame_count % 15] - s_pr[frame_count % 15 - 1]) ** 2
                    S_list.append(delta_s)
                    t_list.append(frame_count)
                    #print('Number of white pixels:', number_of_white_pix)
                    s_w.append(number_of_white_pix)

        else:
            break

    cv2.destroyAllWindows()

    #print(t_list, '\n', S_list)
    
    DF = pd.DataFrame({'t_list':t_list[1:len(t_list)], 'S_list':S_list, 's_w':s_w[1:len(s_w)]})
    
    """timi_t"""
    tt = 0
    timi_t = np.zeros(len(s_w))
    s_ind_list = []

    for s_ind in range(len(s_w[1:len(s_w) - 2])):
        if s_w[s_ind] < s_w[s_ind + 1]:
            tt += 15
            s_ind_list.append(s_ind)
        else:
            timi_t[s_ind_list] = tt * 1.5 / 30
            tt = 0
            s_ind_list = []

    timi_t[timi_t == 0] = mode(timi_t[timi_t != 0], axis = 0)[0]      
    timi_t[timi_t == max(timi_t)] = np.mean(timi_t)
    
    DF['timi_t'] = timi_t[1:len(timi_t)]
    

    """Добавление содержимого в X_train"""
    X_test = X_test.append(DF, ignore_index=True)

Number of white pixels: 288083
Number of white pixels: 284036
Number of white pixels: 284889
Number of white pixels: 285258
Number of white pixels: 283347
Number of white pixels: 287254
Number of white pixels: 290353
Number of white pixels: 295347
Number of white pixels: 319601
Number of white pixels: 295701
Number of white pixels: 335533
Number of white pixels: 318103
Number of white pixels: 322493
Number of white pixels: 322580
Number of white pixels: 323109
Number of white pixels: 324256
Number of white pixels: 318945
Number of white pixels: 321520
Number of white pixels: 303320
Number of white pixels: 220249
Number of white pixels: 191469
Number of white pixels: 198360
Number of white pixels: 189156
Number of white pixels: 177399
Number of white pixels: 172961
Number of white pixels: 177463
Number of white pixels: 175795
Number of white pixels: 175488
Number of white pixels: 167893
Number of white pixels: 369096
Number of white pixels: 348486
Number of white pixels: 356126
Number o

In [17]:
X_test.to_csv('X_test.csv', index=False)

## Нормализация данных

In [16]:
X_test1 = pd.read_csv('X_test.csv')
X_test1

,t_list,S_list,s_w,timi_t
0,30,0.000004,288083,1.50000
1,45,0.000015,284036,1.50000
2,60,0.000012,284889,1.50000
3,75,0.000011,285258,1.50000
4,90,0.000018,283347,1.56457
...,...,...,...,...
145,2205,0.000191,320860,1.50000
146,2220,0.000250,324965,1.50000
147,2235,0.000214,322473,1.50000
148,2250,0.000147,317317,1.50000


In [2]:
"""Импорт таблицы"""
X_test = pd.read_csv('X_test.csv')


In [3]:
for column in X_test.columns:
    X_test[column]=(X_test[column]-X_test[column].min())/(X_test[column].max()-X_test[column].min())
    
X_test.drop(['t_list'], axis=1, inplace = True)

## Выделение обучающих признаков

In [4]:
"""Импорт таблицы"""
X_train = pd.read_csv('X_train.csv')

In [5]:
y = X_train['target']
X_train.drop(['target'], axis=1, inplace = True)

# Обучение модели  
## Нормализация данных

In [6]:
for column in X_train.columns:
    X_train[column]=(X_train[column]-X_train[column].min())/(X_train[column].max()-X_train[column].min())

In [7]:
X_train.drop(['t_list'], axis=1, inplace = True)

## Cлучайный лес 🌳

In [8]:
# Import library
from sklearn.ensemble import RandomForestRegressor

estimator_RF = RandomForestRegressor(n_jobs=-1, random_state=42)

In [9]:
%%time
estimator_RF.fit(X_train, y)

# Test
predictions_RF = estimator_RF.predict(X_test)


CPU times: user 365 ms, sys: 73.1 ms, total: 438 ms
Wall time: 311 ms


In [10]:
predictions_RF

array([ 90.97      ,  92.53      ,  93.36      ,  93.36      ,
        90.26      ,  89.94      ,  90.31      , 123.38      ,
        91.6       , 126.46      , 105.06      ,  91.32      ,
        87.33      ,  87.34      ,  88.68      ,  88.36      ,
        91.29666667,  84.63      , 114.42      ,  86.78      ,
        91.64      ,  88.91      ,  88.91      ,  88.91      ,
        91.65      ,  88.91      ,  88.91      ,  88.91      ,
        91.66      , 103.05      , 106.81      , 103.71      ,
       132.14      ,  89.12      , 108.12      , 135.59      ,
        89.6       ,  91.28      , 123.36      , 135.59      ,
       132.14      ,  95.2       , 106.01      , 134.15      ,
        98.29      , 106.82      , 102.82      ,  89.95      ,
       106.02      ,  89.59      ,  99.2       ,  86.07      ,
       107.47      , 130.94      , 117.44      ,  90.94      ,
       139.9       ,  91.96      , 101.4       ,  91.96      ,
        95.6       ,  94.44      , 101.86      ,  97.85

## XGBoost 🏃🏻

In [11]:
%%time
!pip install xgboost
from xgboost import XGBRegressor
XGB_default = XGBRegressor(random_state=42, tree_method='gpu_hist');

XGB_default.fit(X_train, y_train);

     |████████████████████████████████| 1.2 MB 1.6 MB/s eta 0:00:01


XGBoostError: XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed (vcomp140.dll or libgomp-1.dll for Windows, libomp.dylib for Mac OSX, libgomp.so for Linux and other UNIX-like OSes). Mac OSX users: Run `brew install libomp` to install OpenMP runtime.
  * You are running 32-bit Python on a 64-bit OS
Error message(s): ['dlopen(/Users/nikolay/opt/anaconda3/lib/python3.8/site-packages/xgboost/lib/libxgboost.dylib, 6): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib\n  Referenced from: /Users/nikolay/opt/anaconda3/lib/python3.8/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: image not found']


## Linear Model

In [12]:
# Import library
from sklearn import linear_model

# Model set
estimator1 = linear_model.SGDRegressor(random_state = 42)

In [14]:
%%time
estimator1.fit(X_train, y)
predictions = estimator1.predict(X_test)

CPU times: user 43.2 ms, sys: 9.6 ms, total: 52.8 ms
Wall time: 76.7 ms


In [15]:
predictions_LM

array([116.12890769, 116.94396682, 116.77775469, 116.70492921,
       117.47809884, 116.70338382, 116.04441357, 114.89970455,
       107.48517258, 109.74263693, 101.5673694 , 108.39016173,
       106.89026312, 106.85974065, 106.67348253, 105.86363968,
       103.03655415, 106.82745849, 112.45833651, 120.92754382,
       112.59795706, 118.45315942, 116.82652349, 114.24390239,
       108.45180603, 114.25949394, 113.84765897, 113.77061765,
       107.07173353,  85.6979799 ,  91.31971427,  92.37759994,
       114.10831022, 119.20729451, 115.28495936, 113.37711756,
       120.32811199, 119.63742064, 119.28662015, 113.36215213,
       114.09772769, 109.7027402 , 108.93081436, 113.48661376,
       109.12684012, 108.98914147, 109.09104678, 110.26501946,
       116.50008839, 114.74554513, 111.98797807, 107.09077422,
       108.96512201, 114.28802641, 111.46768155, 107.99309882,
       105.99662864, 107.62036771, 104.87243328, 107.61362352,
       103.95643554, 108.46290708, 104.75133138, 103.52